# Categories  in NL wikipedia

In deze opdracht gaan we een gestructureerde kennisbank maken gebaseerd op de NL wikipedia.

We doen 2 dingen

1. We maken het netwerk (het is geen boom maar een DAG) van categorieen.
2. Voor elke pagina, geven we aan 
    * onder welke categorieen het _rechtstreeks_ valt
     

### Categorie-informatie
* Voor elke pagina kunnen we de verzameling categorieen die rechtstreeks bij die pagina horen (die dus onder aan de pagina staan) vergelijken met de categorieen die we ook met beghulp van overerving zouden krijgen.
    * Staat er bijvoorbeeld vaak redundante informatie op paginas? 
    * Hoe verhouden het aantal  expliciete en impliciete categorieen zich?
    
 

## Categories



### Doel
* We willen een netwerk van categori&euml;en distelleren uit de NL wikipedia.
* Dat bestaat en kunnen we raadplegen via <https://nl.wikipedia.org/wiki/Speciaal:Categorieboom>
* Nu willen we die data zelf hebben en in een kennisbank opslaan.

#### Hoe kunnen we dat doen?

##### Methode 1: extrinsiek
1. Download Wikipedia
2. Bekijk voor  elke categorie, welke pagina die heeft.
3. De _deelverzameling_ relatie is dan de _"is-bevat-in"_, of _is-specifieker-dan_ relatie.

##### Methode 2: intrinsiek
1. Probeer de structuur van de categorieen ergens binnen Wikipedia te vinden.
2. Hier is iemand die hetzelfde wilde: <http://stackoverflow.com/questions/12227134/how-to-fetch-category-tree-of-wiki>

### Dumps/data
* Dumps: <https://dumps.wikimedia.org/nlwiki/20150602/>
* API: <https://www.mediawiki.org/wiki/API%3aCategorymembers>
* Je bent niet de eerste met deze opdracht ....
    * Zoek op het web.
    * Bijvoorbeeld: <http://stackoverflow.com/questions/8088226/content-of-infobox-of-wikipedia/21107068#21107068>
    * Zie ook <https://www.mediawiki.org/wiki/Alternative_parsers> voor nog meer mediawiki parsers


# Opdrachten

## Categorieen DAG

### Download en representeer. 30%
1. Verzin een datastructuur voor de categorieen DAG. Haal de NL wikipedia categorie structuur binnen, en sla die op in je datastructuur.

### Analyseer 30%
1. Hoeveel unieke categorieen zijn er?
1. Hoe diep is de DAG? Wat is het langste pad van de root naar een blad, en wat is het kortste?
2. Maak plotjes waarin je voor elk getal `n` aangeeft hoeveel categorieen er zijn met 
    1.  `n` directe ouders
    2. `n` voorouders (let op, je werkt in een DAG, niet in een boom)
    3. exact `n` kinderen.
3. Welke categorie heeft de meeste kinderen?
4. Welke de meeste "nakomelingen"?

## Categorieen en paginas 30%
1. Bepaal voor elke Wikipedia pagina tot welke categorieen die rechtstreeks behoort, en tot welke categorieen die behoort dmv overerving.
    * Kies een slimme manier om dit te representeren.
1. Bepaal voor elke Wikipedia pagina, hoeveel van de expliciet genoemde categorieen, ook via overerving hadden kunnen worden afgeleid.
    * Wat is het gemiddelde per pagina?
    
## Reflecteer 10%
Beschrijf in een aantal punten wat er makkelijk was en wat lastig was aan de opdracht. 
Geef goed weer wat voor technieken en mogelijkheden je hebt gebruikt.

# Wat probeersels  (negeer dit gerust)

In [2]:
#!mkdir NLWikipedia
%cd NLWikipedia/

/Users/admin/Documents/work/onderwijs/teaching/KennisSystemen/Assignments/NLWikipedia


## Category tabel

In [10]:
#! wget --no-check-certificate https://dumps.wikimedia.org/nlwiki/20150602/nlwiki-20150602-category.sql.gz
#!gunzip nlwiki-20150602-category.sql.gz
#!head -100 nlwiki-20150602-category.sql
!gzip nlwiki-20150602-category.sql

gzip: can't stat: nlwiki-20150602-category.sql (nlwiki-20150602-category.sql): No such file or directory


## mmm

* Aan bovenstaande file heb je niet veel, want je krijgt de categorie structuur niet! 
* Kijk maar naar het tabelformaat en de voorbeelden:

```
CREATE TABLE `category` (
  `cat_id` int(10) unsigned NOT NULL AUTO_INCREMENT,
  `cat_title` varbinary(255) NOT NULL DEFAULT '',
  `cat_pages` int(11) NOT NULL DEFAULT '0',
  `cat_subcats` int(11) NOT NULL DEFAULT '0',
  `cat_files` int(11) NOT NULL DEFAULT '0',
  PRIMARY KEY (`cat_id`),
  UNIQUE KEY `cat_title` (`cat_title`),
  KEY `cat_pages` (`cat_pages`)
) ENGINE=InnoDB AUTO_INCREMENT=14627193 DEFAULT CHARSET=binary;
/*!40101 SET character_set_client = @saved_cs_client */;

--
-- Dumping data for table `category`
--

/*!40000 ALTER TABLE `category` DISABLE KEYS */;
INSERT INTO `category` VALUES (1,'Plaats_in_Portugal',3577,6,0),(2,'Wikipedia:Afbeelding_gewenst',126,7,0),(3,'Spionageroman',16,2,0),(4,'Ridderorde_in_Polen',16,1,0),(5,'Nederlands_kunstenaar',481,24,0),(6,'Nederlands_kunstschilder',2203,5,0),(7,'Wikipedia:Werk_in_uitvoering',17,0,0),(8,'South_Park',17,2,0)
```

* Als we de tabel vergelijken met bijvoorbeeld <https://nl.wikipedia.org/wiki/Albert_Speer> zien we dat er alleen instaat welke categorien elke pagina bevat.
* We hebben nog niet de boom zelf.

In [12]:
# We kunnen ook alle artikelen ophalen.
# Dit hier is de grootste: er zijn ook veel kleinere. Begin daar liever mee te werken.

!wget --no-check-certificate  https://dumps.wikimedia.org/nlwiki/20150602/nlwiki-20150602-pages-articles.xml.bz2

--2015-07-27 15:19:14--  https://dumps.wikimedia.org/nlwiki/20150602/nlwiki-20150602-pages-articles.xml.bz2
Resolving dumps.wikimedia.org... 208.80.154.11, 2620::861:1:208:80:154:11
Connecting to dumps.wikimedia.org|208.80.154.11|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 1080732485 (1.0G) [application/octet-stream]
Saving to: `nlwiki-20150602-pages-articles.xml.bz2'

100%[====================================>] 1,080,732,485 1.21M/s   in 13m 23s 

2015-07-27 15:32:37 (1.28 MB/s) - `nlwiki-20150602-pages-articles.xml.bz2' saved [1080732485/1080732485]



In [73]:
# This is a huge file, we just want to inspect it now, and so only decompress the first 10.000 lines
# Run this in the shell. From here you get an error
!bzcat "nlwiki-20150602-pages-articles.xml.bz2" |head -10000 > small.xml


bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = nlwiki-20150602-pages-articles.xml.bz2, output file = (stdout)


In [5]:
!head -80 NLWikipedia/small.xml



<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.10/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.10/ http://www.mediawiki.org/xml/export-0.10.xsd" version="0.10" xml:lang="nl">
  <siteinfo>
    <sitename>Wikipedia</sitename>
    <dbname>nlwiki</dbname>
    <base>http://nl.wikipedia.org/wiki/Hoofdpagina</base>
    <generator>MediaWiki 1.26wmf7</generator>
    <case>first-letter</case>
    <namespaces>
      <namespace key="-2" case="first-letter">Media</namespace>
      <namespace key="-1" case="first-letter">Speciaal</namespace>
      <namespace key="0" case="first-letter" />
      <namespace key="1" case="first-letter">Overleg</namespace>
      <namespace key="2" case="first-letter">Gebruiker</namespace>
      <namespace key="3" case="first-letter">Overleg gebruiker</namespace>
      <namespace key="4" case="first-letter">Wikipedia</namespace>
      <namespace key="5" case="first-letter">Overleg Wikipedia</nam

### Extracting information with Beautifulsoup

* Dit is een makkelijke Python module om met HTML en XML fies om te gaan.
* Hieronder zie je wat voorbeelden

### Extracting information with lxml
* Dit werkt net wat lastiger maar is wel veel stabieler en kan ook joekels van files aan.
* Je hoeft zelfs een file niet eerst te "ontzippen", maar kan hem stromend in lxml inlezen.
* Dat betekent dat je niet snel geheugen problemen zal hebben: want jij hoeft maar steeds 1 pagina in geheugen te hebben.
* Dit is een deel van mijn antwoord:
* Je ziet dat je rekening moet houden met de namespaces (dat hoeft in Beautifulsoup niet).
* Alles werkt net wat lastiger dan in BeautifulSoup, maar het schaalt wel.

```
from lxml import etree as et
from bz2file import BZ2File

def create_page_category_dict(f):
    with BZ2File(f) as xml_file:
        context = etree.iterparse(xml_file)
        
        pages_categories_dict={} 
        for action, elem in context:
             if elem.tag== '{http://www.mediawiki.org/xml/export-0.10/}page':
                title=  
                pagetext=  
                cats= find_categories(pagetext) # own function which extract the categories from the text of a page using a regular expression
                pages_categories_dict[title]=cats # update the dict
        return pages_categories_dict

# call it (it took 1 minute on my macbook pro)
%time ef= create_page_category_dict('nlwiki-20150602-pages-articles1.xml.bz2')
```

In [6]:
# http://www.crummy.com/software/BeautifulSoup/bs4/doc/
from bs4 import BeautifulSoup
small= open("NLWikipedia/small.xml")
soup = BeautifulSoup(small)

In [7]:
# list first ten  pages in our small dump
soup.findAll('title')[:10]

[<title>Albert Speer</title>,
 <title>Andre Agassi</title>,
 <title>Groen (partij)</title>,
 <title>Anthony Fokker</title>,
 <title>Albert Plesman</title>,
 <title>Alan Turing</title>,
 <title>Algoritme</title>,
 <title>Alpen</title>,
 <title>Antwerpen (stad)</title>,
 <title>Astronomie</title>]

In [40]:
pages= soup.findAll('page')

page_infobox_dict = {p.title.text:p.find('text') for p in pages}
   
speertext=page_infobox_dict['Albert Speer']

speertext.text[:2000]

u'{{Zie artikel|Zie [[Albert Speer (junior)|Albert Speer jr.]] voor de zoon van Albert Speer, ook een Duits planoloog.}}\n{{Infobox persoon Tweede Wereldoorlog\n| naam=Albert Speer\n| afbeelding=Albert Speer Neurenberg.JPG\n| onderschrift=Speer op het [[Proces van Neurenberg]] (1945-1946)\n| gbdat=19 maart 1905\n| sterfdat=1 september 1981\n| geslacht=Man\n| gbplaats=Mannheim\n| sterfplaats=Londen\n| zijde=Nazi-Duitsland\n| org=NSDAP\n| spfunc=Minister van Bewapening en Munitie\n| rang=\n}}\n\'\'\'Berthold Konrad Hermann Albert Speer\'\'\' ([[Mannheim]], [[19 maart]] [[1905]] \u2013 [[Londen]], [[1 september]] [[1981]]) was een [[Duitsland|Duits]] [[architect]] en tijdens de [[nationaalsocialisme|nazi-heerschappij]] over Duitsland (1933-1945) achtereenvolgens rijksbouwmeester en minister van bewapening. Door zijn vriendschap met de dictator [[Adolf Hitler]] en zijn ministerschap gold hij als een van de machtigste mannen van het [[nazi-Duitsland|Derde Rijk]].<ref>[http://www.conne-islan